In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

import cv2
from PIL import Image
import random
from albumentations import *
from tqdm import tqdm

In [ ]:
AUG = [ HorizontalFlip, ShiftScaleRotate, ]
print(len(AUG))

In [ ]:
path0="../input/100-bird-species/birds/test/AMERICAN GOLDFINCH/4.jpg"
image0c=cv2.imread(path0)
aug=AUG[0](p=1)
image2c=aug(image=image0c)['image']
plt.figure(figsize=(4,4))
plt.imshow(cv2.cvtColor(image2c, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
image0=load_img(path0, grayscale=False, color_mode='rgb', target_size=(32,32))
image1=np.asarray(image0)
aug=AUG[0](p=1)
image2=aug(image=image1)['image']
plt.figure(figsize=(4,4))
plt.imshow(image2)
plt.show()

In [ ]:
train_dir = '../input/100-bird-species/birds/train/'
test_dir = '../input/100-bird-species/birds/test/'

In [ ]:
Name=[]
for file in os.listdir(train_dir):
    Name+=[file]
print(Name)
print(len(Name))

In [ ]:
N=[]
for i in range(len(Name)):
    N+=[i]

normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [ ]:
trainset=[]
trainlabel=[]
count=0
for file in tqdm(os.listdir(train_dir)):
    path=os.path.join(train_dir,file)
    for im in os.listdir(path):
        image0=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(32,32))
        image1=np.asarray(image0)
        for i in range(len(AUG)):
            aug=AUG[i](p=1)
            image2=aug(image=image1)['image']       
            image3=image2/255.0
            if image3.sum()>20:
                trainset+=[image3]
                trainlabel+=[count]        
    count=count+1

In [ ]:
testset=[]
testlabel=[]
count=0
for file in tqdm(os.listdir(test_dir)):
    path=os.path.join(test_dir,file)
    for im in os.listdir(path):
        image0=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(32,32))
        image1=np.asarray(image0)
        for i in range(len(AUG)):
            aug=AUG[i](p=1)
            image2=aug(image=image1)['image']       
            image3=image2/255.0
            if image3.sum()>20:
                testset+=[image3]
                testlabel+=[count]        
    count=count+1

In [ ]:
trainX=np.array(trainset)
testX=np.array(testset)
trainY=np.array(trainlabel)
testY=np.array(testlabel)

In [ ]:
m=len(trainset)
M=[]
for i in range(m):
    M+=[i]
random.seed(2021)
random.shuffle(M)

In [ ]:
fig,axs = plt.subplots(6,6,figsize=(15,15))
for i in range(36):
    image=trainset[M[i]]
    r=i//6
    c=i%6
    axs[r][c].set_xticks([])
    axs[r][c].set_yticks([])
    ax=axs[r][c].imshow(image)
plt.show()

In [ ]:
trainlabels1=to_categorical(trainY)
trainlabels=np.array(trainlabels1)

In [ ]:
trainx,testx,trainy,testy=train_test_split(trainX,trainlabels,test_size=0.2,random_state=44)

In [ ]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [ ]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(32,32,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(275, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=30)

In [ ]:
testlabels1=to_categorical(testY)
testlabels=np.array(testlabels1)

In [ ]:
y_pred=model.predict(testX)
pred=np.argmax(y_pred,axis=1)
ground=np.argmax(testlabels,axis=1)
print(classification_report(ground,pred))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
path0="../input/100-bird-species/birds/test/AMERICAN GOLDFINCH/4.jpg"
load_img(path0,target_size=(200,200))

In [ ]:
image=load_img(path0,target_size=(32,32))
image=img_to_array(image) 
image=image/255.0
prediction_image=np.expand_dims(image, axis=0)
print(prediction_image.shape)

In [ ]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=reverse_mapping[value]
print("Prediction is {}.".format(move_name))

In [ ]:
pred2=model.predict(testX)

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]
print(PRED[0:10])

In [ ]:
ANS=list(testY)
print(ANS[0:10])

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)